In [90]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [91]:
import numpy as np 
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

# **QUESTION 1**

Importing the data, having initial look on the dataset and looking for null values.

In [92]:
df = pd.read_csv('/kaggle/input/task-rapp/ingredient.csv', delimiter=',')
df.head()

In [93]:
df.isnull().sum()

The dataset is clear from null values, and proceed with further exploration

## **QUESTION 1a. (Descriptive Anlysis,Correlation and Anova)**

### **Descriptive Analysis**

In [94]:
df.describe()

**Findings:**
* ingredient a, e have relatively low standard deviation compared to it's means *(Could be eliminated)*
* ingredient a, b, d, e, g is a must-have in the formula
* ingredient c, f ,h ,i are optionals since it has a minimum value of zero *(Couldn't be eliminated since it's important for clustering)*
* some ingredients have higher means compared to the rest *(Normalized data needed)*

To see better on how each standard deviation interacts with each ingredient respective mean, I normalized the data by MinMax method

In [95]:
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df.describe()

**Findings:**
* h and i has high concentration in the near-zero value, eavluated from it's Q1, Q2 and Q3 value *(Might need some kind of transformation)* 

### **Correlation Analysis**

In [96]:
corr = normalized_df.corr()
corr.style.background_gradient(cmap='coolwarm')
corr.style.background_gradient(cmap='coolwarm').set_precision(2)
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

**Findings:** 
From the correlation plot, I would like to classify each relations into some groups:
* High Positive Correlation (>0,75):
1. a and g
* High Negative Correlation (<-0,75):
1. None
* Moderate Positive Correlation (0,25<x<0,75):
1. b and h
1. d and f
1. d and h
* Moderate Negative Correlation (-0,75<x<-0,25):
1. a and d
1. a and e
1. a and f
1. b and c
1. b and f
1. b and g
1. c and d
1. c and g
1. c and h
1. d and g
* And the rest could be classified as uncorrelated


* High negatively correlated ingredients might mean interchangeable subtances on the formula *(eventhough they're interchangeable they could be clustered into two different groups, so I chose to retain the data with this kind of correlation)*
* High positively correlated ingredients might mean redundancy on the factors/variables and can be reduced to a single variable *(I eliminate a since it has high correlation with g, strengthening the previous argument to eliminate a)*


## **QUESTION 1b. (Graphical Analysis and Distribution Study)**

### **Graphical Analysis**

Making density plots and histograms of each variable

In [97]:
fig, axs = plt.subplots(2, 5, figsize=(21, 7))

sns.histplot(data=normalized_df, x="a", kde=True,color="skyblue", ax=axs[0, 0])
sns.histplot(data=normalized_df, x="b", kde=True,color="olive", ax=axs[0, 1])
sns.histplot(data=normalized_df, x="c", kde=True,color="gold", ax=axs[0, 2])
sns.histplot(data=normalized_df, x="d", kde=True,color="teal", ax=axs[0, 3])
sns.histplot(data=normalized_df, x="e", kde=True,color="red", ax=axs[0, 4])
sns.histplot(data=normalized_df, x="f", kde=True,color="green", ax=axs[1, 0])
sns.histplot(data=normalized_df, x="g", kde=True,color="yellow", ax=axs[1, 1])
sns.histplot(data=normalized_df, x="h", kde=True,color="pink", ax=axs[1, 2])
sns.histplot(data=normalized_df, x="i", kde=True,color="blue", ax=axs[1, 3])

plt.show()

Making boxplots of each variable

In [98]:
fig, axs = plt.subplots(2, 5, figsize=(21, 7))

sns.boxplot(data=normalized_df, y="a",color="skyblue", ax=axs[0, 0])
sns.boxplot(data=normalized_df, y="b",color="olive", ax=axs[0, 1])
sns.boxplot(data=normalized_df, y="c",color="gold", ax=axs[0, 2])
sns.boxplot(data=normalized_df, y="d",color="teal", ax=axs[0, 3])
sns.boxplot(data=normalized_df, y="e",color="red", ax=axs[0, 4])
sns.boxplot(data=normalized_df, y="f",color="green", ax=axs[1, 0])
sns.boxplot(data=normalized_df, y="g",color="yellow", ax=axs[1, 1])
sns.boxplot(data=normalized_df, y="h",color="pink", ax=axs[1, 2])
sns.boxplot(data=normalized_df, y="i",color="blue", ax=axs[1, 3])

plt.show()


Making scatterplot matrix

In [99]:
sns.pairplot(normalized_df)

**Findings:** 

From the density and histogram plots
* visually the ingredients a, b, d, and g are right-skewed
* visually the ingredient e is left-skewed *(The skewed ones are fine but transformation is needed to reduce their tails)*
* visually the ingredients h and i are zero-inflated *(This property is important in this problem so I let them be)*
* visually the ingredient c and f is zero-inflated and bi-modal *(This property is important in this problem so I let them be)*

From the boxplots and pair plots
* almost all of the ingredients has notable outliers *(Getting rid of the outliers as it may affect the analysis. e.g. as in ingredient f where the bi-modality nature of the data wasn't showing due to outliers)*
* in b >0.8 
* in d >0.8
* in f >0.4
* in h >0.6
* in i >0.8

## **QUESTION 1c. (Unsupervised Clustering)**

### **Transforming the Dataset**

From what we had explored on the previous questions, there some notable properties that we got:
* some ingredients have higher means compared to the rest ->*Normalized dataset is needed to eliminate biases on distance measurements*
* ingredient c, f ,h ,i are optionals since it has a minimum value of zero -> *This variables can't be reduced since the nature of dataset made zero inflated variable even more valuable*
* ingredients a and e have a low standard deviation -> *they can be eliminated since there are no relevant changes in quantity*
* a and g have high positive correlation -> *one of them could be eliminated to eliminate redundancy*
* visually the ingredients h and i are zero-inflated  -> *this variables can't be reduced since the nature of dataset made zero inflated variable even more valuable*
* visually the ingredient c and f are bi-modal and zero-inflated -> *no transformation needed since the nature of dataset made bi-modal variable even more valuable*
* almost all of the ingredients had lots of outliers -> *this might be due to the tweaking of contents, reducing some of the data is needed to strengthen some characteritics that a feature has*



**Transformation that is going to be done:**
* Eliminate ingredient a and e 
* Normalized the data by it's Min and Max value to retain the zero inflated-ness
* Eliminate some of the data that's an outlier

In [100]:
# using normalized_df instead of df
# eliminating ingredients a and e
normalized_df = normalized_df.drop(['a','e'],axis=1)
# filtering the data from outliers
normalized_df = normalized_df[normalized_df['b'] <= 0.8]
normalized_df = normalized_df[normalized_df['d'] <= 0.8]
normalized_df = normalized_df[normalized_df['f'] <= 0.15]
normalized_df = normalized_df[normalized_df['g'] <= 0.8]
normalized_df = normalized_df[normalized_df['h'] <= 0.6]
normalized_df = normalized_df[normalized_df['i'] <= 0.8]

In [101]:
normalized_df=(normalized_df-normalized_df.min())/(normalized_df.max()-normalized_df.min())
normalized_df.info()

In [102]:
sns.pairplot(normalized_df)

**Findings after Transformation:** 
Seems like the dataset is where i wanted it to be, and I'll proceed with the clustering

### **Using K-Means Clustering and Elbow Method to find number of clusters**

In [103]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


x_array = np.array(normalized_df)
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x_array)

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(x_scaled)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()


**Finding:** k=5 is visually being the elbow, I'll use 5 cluster

### **Using the Dendogram to get intuitive with the result**

In [104]:
from sklearn.preprocessing import normalize
data_scaled = normalize(normalized_df)
data_scaled = pd.DataFrame(data_scaled, columns=normalized_df.columns)
import scipy.cluster.hierarchy as shc
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(data_scaled, method='ward'))
plt.axhline(y=2, color='r', linestyle='--')

**Findings:** Seems like the dendogram goes hand in hand with the KMeans, I'll continue to group them into 5 clusters

### **Conclusion**

Finding the centroids of each cluster to have better look on what properties does each cluster has

In [105]:
kmeans = KMeans(n_clusters = 5, random_state=123)
kmeans.fit(x_scaled)
cc = kmeans.cluster_centers_
print(cc)

In [106]:
df_cc = pd.DataFrame(cc, columns=['b','c','d','f','g','h','i'])
df_cc

**Final Findings:**



From the centroid data above, I classify each value qualitatively by each ingredients relative presence on the formula as 
* 0.30<= would be low, 
* 0.30< x <0.6 is moderate  
* =>0.60 is High
* and <=0.1 is absent

And here is what the end result looks like

In [107]:
con = {'Ing.b':['Moderate','High','Moderate','Moderate','Moderate'],'Ing.c':['High','Absent','High','Absent','High'],'Ing.d':['Moderate','High','Low','Moderate','Moderate'],'Ing.f':['High','Absent','Low','Moderate','High'],'Ing.g':['Low','Low','Moderate','High','Low'],'Ing.h':['Absent','High','Absent','Absent','Absent'],'Ing.i':['Moderate','Absent','Low','Low','Absent']} 
condf = pd.DataFrame(con)
condf

# **QUESTION 2**

Importing and early look at the data

In [108]:
df2 = pd.read_csv('/kaggle/input/task-rapp/palm_ffb.csv', delimiter=',')
df2.dataframeName = 'palm_ffb.csv'
nRow, nCol = df2.shape
print(f'There are {nRow} rows and {nCol} columns')

In [109]:
df2.head(5)

In [110]:
df2.isnull().sum()

**Findings:** No null values in the dataset, Since it's a yield data the date may affect the value of the yield due to seasonal changes. For this I then took the month of the year to replace the current date format

## **Descriptive and Graphical Analysis**

### Descriptive Analysis

In [111]:
df2.describe()

### Graphical Analysis

In [112]:
fig, axs = plt.subplots(2, 4, figsize=(21, 7))

sns.histplot(data=df2, x="SoilMoisture", kde=True,color="skyblue", ax=axs[0, 0])
sns.histplot(data=df2, x="Average_Temp", kde=True,color="olive", ax=axs[0, 1])
sns.histplot(data=df2, x="Min_Temp", kde=True,color="gold", ax=axs[0, 2])
sns.histplot(data=df2, x="Max_Temp", kde=True,color="teal", ax=axs[0, 3])
sns.histplot(data=df2, x="Precipitation", kde=True,color="red", ax=axs[1, 0])
sns.histplot(data=df2, x="Working_days", kde=True,color="green", ax=axs[1, 1])
sns.histplot(data=df2, x="HA_Harvested", kde=True,color="yellow", ax=axs[1, 2])
sns.histplot(data=df2, x="FFB_Yield", kde=True,color="pink", ax=axs[1, 3])

plt.show()

In [113]:
fig, axs = plt.subplots(2, 4, figsize=(21, 7))

sns.boxplot(data=df2, y="SoilMoisture",color="skyblue", ax=axs[0, 0])
sns.boxplot(data=df2, y="Average_Temp",color="olive", ax=axs[0, 1])
sns.boxplot(data=df2, y="Min_Temp",color="gold", ax=axs[0, 2])
sns.boxplot(data=df2, y="Max_Temp",color="teal", ax=axs[0, 3])
sns.boxplot(data=df2, y="Precipitation",color="red", ax=axs[1, 0])
sns.boxplot(data=df2, y="Working_days",color="green", ax=axs[1, 1])
sns.boxplot(data=df2, y="HA_Harvested",color="yellow", ax=axs[1, 2])
sns.boxplot(data=df2, y="FFB_Yield",color="pink", ax=axs[1, 3])

plt.show()

In [114]:
sns.pairplot(df2)

**Findings:**
* Most of variables except for FFB_Yield isn't really skewed *(Further transformation might not be needed)*
* Precipitation, Min_Temp, Working_days and HA_Harvested have high Kurtosis *(Getting rid of the otliers that causes the kurtosis)*
* Precipitation, Min_Temp, and HA_Harvested have noticable outliers *(Getting rid of the outliers)*
* The features is suffering from collinearity or multi-collinearity *(Use VIF)*


### Corellation Plot

In [115]:
corr = df2.corr()
corr.style.background_gradient(cmap='coolwarm')
corr.style.background_gradient(cmap='coolwarm').set_precision(2)
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

**Findings:** 
From the correlation plot, I would like to classify each relations into some groups:
* High Positive Correlation (>0,75):
1. Max_Temp and Average_Temp
* High Negative Correlation (<-0,75):
1. None
* Moderate Positive Correlation (0,25<x<0,75):
1. Precipitation and Soil Moisture
1. HA_Harvested and Average_Temp
1. FFB_Yield and Precipitation
1. Precipitation and Min_Temp
1. HA_Harvested and Max_Temp
* Moderate Negative Correlation (-0,75<x<-0,25):
1. Average_Temp and Soil_Moisture
1. Soil_Moisture and Max_Temp
1. Precipitation and Average_Temp
1. Precipitation and Max_Temp
1. HA_Harvested and Soil_Moisture
1. HA_Harvested and Precipitation
* And the rest could be classified as uncorrelated
* The features is suffering from collinearity or multi-collinearity *(Use VIF)*

## **Feature Engineering**
### Removing the outliers

In [116]:
#Removing Outliers
df2 = df2[df2['Min_Temp']>=19.8]
df2 = df2[df2['Precipitation']<=350]
df2 = df2[df2['HA_Harvested']>=700000]

Another quick look to the distplot to see whether the removal of outlliers helped

In [117]:
fig, axs = plt.subplots(2, 4, figsize=(21, 7))

sns.histplot(data=df2, x="SoilMoisture", kde=True,color="skyblue", ax=axs[0, 0])
sns.histplot(data=df2, x="Average_Temp", kde=True,color="olive", ax=axs[0, 1])
sns.histplot(data=df2, x="Min_Temp", kde=True,color="gold", ax=axs[0, 2])
sns.histplot(data=df2, x="Max_Temp", kde=True,color="teal", ax=axs[0, 3])
sns.histplot(data=df2, x="Precipitation", kde=True,color="red", ax=axs[1, 0])
sns.histplot(data=df2, x="Working_days", kde=True,color="green", ax=axs[1, 1])
sns.histplot(data=df2, x="HA_Harvested", kde=True,color="yellow", ax=axs[1, 2])
sns.histplot(data=df2, x="FFB_Yield", kde=True,color="pink", ax=axs[1, 3])

plt.show()

It helped normalizing the data

### Calculating VIF Score

I then calculate the VIF value to make sure if there are any features that are heavily correlated to one or more features that might reduce the performance of the model

In [118]:
#Calculating VIF and making a Dataframe to visualize
import pandas as pd
from sklearn.linear_model import LinearRegression
def calculate_vif(df, features):    
    vif, tolerance = {}, {}
    for feature in features:
        X = [f for f in features if f != feature]        
        X, y = df[X], df[feature]
        r2 = LinearRegression().fit(X, y).score(X, y)                
        
   
        tolerance[feature] = 1 - r2
        vif[feature] = 1/(tolerance[feature])
    return pd.DataFrame({'VIF': vif, 'Tolerance': tolerance})

In [119]:
calculate_vif(df2, features=['SoilMoisture','Average_Temp','Min_Temp','Max_Temp',
                                'Precipitation','Working_days','HA_Harvested'])

**Findings:** The average temperature features is highly correlated to more than one other features in the dataset, and also the VIF value is pretty high. Hence, I remove this feature

In [120]:
calculate_vif(df2, features=['SoilMoisture','Min_Temp','Max_Temp',
                                'Precipitation','Working_days','HA_Harvested'])

**Findings:**
Almost all of the features are now having relatively low VIF value compared to their initial values, for now I consider them as independent variables and could be used for further analysis 

## **Modelling**

### Linear Regression
* Dropping Average_Temp
* Normalizing the data by MinMax so there won't be any scalability issues with linear regression coefficients

In [121]:
df2_1 = df2.drop("Average_Temp", axis=1)
df2_1 = df2_1.drop("Date", axis=1)
normdf2_1 = (df2_1 - df2_1.min())/(df2_1.max()-df2_1.min())
normdf2_2 = normdf2_1
normdf2_1 = pd.concat([df2['Date'],normdf2_1],axis=1)
normdf2_1['Month'] = pd.DatetimeIndex(normdf2_1['Date']).day
normdf2_1 = normdf2_1.drop("Date", axis=1)
normdf2_1=pd.get_dummies(normdf2_1, columns=["Month"])

Making model, and fitting the data

In [122]:
from sklearn.linear_model import LinearRegression
X=normdf2_2.drop('FFB_Yield', axis=1)
y=normdf2_2['FFB_Yield']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10, random_state=101)
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

print("The accuracy score of the model:", lin_reg.score(X_test, y_test))

**Findings:**
The accuracy score is bad, I'll use other method of regression and use this result as a base

Finding feature importance from the linear regression model displaying it in table and graph

In [123]:
importance_lin_reg = lin_reg.coef_
# summarize feature importance
feature_importances_lin_reg = pd.DataFrame(importance_lin_reg,index = X_train.columns,
                                 columns=['importance']).sort_values('importance',ascending=False)
feature_importances_lin_reg

In [124]:
from matplotlib import pyplot
pyplot.bar(X_train.columns, importance_lin_reg)
pyplot.show()

**Findings:**
* The 3 features with highest importance are: Precipitation, HA_Harvested and SoilMoisture

### Random Forest Regressor

Random forest is well known to be pretty robust but tend to overfit. To cover for the lack of accuracy from the linear regression model, I'll use Random Forest Regressor

Side note: I also added month as a feature since i believe that it might be useful

In [125]:
normdf2_1.head()

Putting the importance score into a table and a graph

In [126]:
# random forest for feature importance on a regression problem
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot

X1=normdf2_1.drop('FFB_Yield', axis=1)
y1=normdf2_1['FFB_Yield']

from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1,y1,test_size=0.30, random_state=101)


# define the model
rf = RandomForestRegressor()
# fit the model
model = rf.fit(X1, y1)
# get importance
importance = rf.feature_importances_
# summarize feature importance
feature_importances = pd.DataFrame(importance,index = X_train1.columns,
                                 columns=['importance']).sort_values('importance',ascending=False)
feature_importances

In [127]:
pyplot.bar(X_train1.columns, importance)
pyplot.show()

In [128]:
print ("This model has an accuracy score of:", model.score(X_test1, y_test1))

**Findings:**
* The 3 features with highest importance are: Precipitation, HA_Harvested and SoilMoisture
* Month of the year is also showing pretty interesting pattern as some months are really affecting the yields and some doesn't
* for the top features that affects the yield, the Random Forest Regressor agrees with the Linear Model which is nice


## **Conclusion**
**Final Findings:**
* The 3 features with highest importance score: Precipitation, HA_Harvested and SoilMoisture
* Precipitation is positively correlated with yield and the other two are negatively correlated
* HA_Harvested being important is strange, since the FFB_Yield has a unit of Tonnes/HA

# **QUESTION 3**

## **Preliminary**

Putting the pharagraph into a string

In [129]:
string = """As a term, data analytics predominantly refers to an assortment of applications, from basic business 
intelligence (BI), reporting and online analytical processing (OLAP) to various forms of advanced
analytics. In that sense, it's similar in nature to business analytics, another umbrella term for
approaches to analyzing data -- with the difference that the latter is oriented to business uses, while
data analytics has a broader focus. The expansive view of the term isn't universal, though: In some
cases, people use data analytics specifically to mean advanced analytics, treating BI as a separate
category.  Data analytics initiatives can help businesses increase revenues, improve operational
efficiency, optimize marketing campaigns and customer service efforts, respond more quickly to
emerging market trends and gain a competitive edge over rivals -- all with the ultimate goal of
boosting business performance. Depending on the particular application, the data that's analyzed
can consist of either historical records or new information that has been processed for real-time
analytics uses. In addition, it can come from a mix of internal systems and external data sources.  At
a high level, data analytics methodologies include exploratory data analysis (EDA), which aims to find
patterns and relationships in data, and confirmatory data analysis (CDA), which applies statistical
techniques to determine whether hypotheses about a data set are true or false. EDA is often
compared to detective work, while CDA is akin to the work of a judge or jury during a court trial -- a
distinction first drawn by statistician John W. Tukey in his 1977 book Exploratory Data Analysis.  Data
analytics can also be separated into quantitative data analysis and qualitative data analysis. The
former involves analysis of numerical data with quantifiable variables that can be compared or
measured statistically. The qualitative approach is more interpretive -- it focuses on understanding
the content of non-numerical data like text, images, audio and video, including common phrases,
themes and points of view."""

In [130]:
string

Cleaning the string from unwanted characters, such as line-breaks (\n), punctuation etc.

In [131]:
string = string.replace("\n"," ")
string = string.replace("--","")
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
for x in string:
    if x in punc:
        string = string.replace(x, "")
string 

Seems like we're good to go

## **Question 3a.**
counting number of words in the paragraph 

In [132]:
x = len(string.split())
print("Number of word in the paragraph:", x)

In [133]:
y = string.count('data')
a = (y/x)
print("Probability of word 'data':","{:.2%}".format(a))

## **Question 3b.**

In [134]:
z=len(set(string.split()))
b=(z/x)
print("Probability of unique word used in the paragraph:","{:.2%}".format(b))

## **Question 3c.**

In [135]:
c = string.count('data analytics')
d = string.count('analytics')
e = (c/d)
print("Probability of 'analytics' occuring after 'data':","{:.2%}".format(e))